In [1]:
from pyspark import SparkContext

In [2]:
sc = SparkContext(master='local', appName= 'tranformacionesYAcciones')

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/12/02 12:19:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
rdd = sc.parallelize([1,2,3])
type(rdd)

pyspark.core.rdd.RDD

In [4]:
rdd.collect()

[1, 2, 3]

In [5]:
sc

<SparkContext master=local appName=tranformacionesYAcciones>

In [6]:
!ls /home/jovyan/data/

deporte.csv	 deportistaError.csv  modelo_relacional.jpg
deportista2.csv  evento.csv	      paises.csv
deportista.csv	 juegos.csv	      resultados.csv


In [7]:
!ls /home/jovyan/data

deporte.csv	 deportistaError.csv  modelo_relacional.jpg
deportista2.csv  evento.csv	      paises.csv
deportista.csv	 juegos.csv	      resultados.csv


In [8]:
path = '/home/jovyan/data/'

In [9]:
from pyspark.sql import SparkSession

In [10]:
equiposOlimpicosRDD = sc.textFile(path+"paises.csv") \
.map(lambda line : line.split(","))

In [11]:
equiposOlimpicosRDD.take(5)


In [11]:
equiposOlimpicosRDD.map(lambda x: x[2]).distinct().count()

231

In [12]:
equiposOlimpicosRDD.map(lambda x: (x[2], x[1])).groupByKey().mapValues(len).take(5)

[('sigla', 1), ('AUT', 11), ('MEX', 9), ('ARG', 18), ('AFG', 1)]

In [13]:
equiposOlimpicosRDD.map(lambda x: (x[2], x[1])).groupByKey() \
    .mapValues(list).take(5)

[('sigla', ['equipo']),
 ('AUT',
  ['30. Februar',
   'Austria',
   'Austria-1',
   'Austria-2',
   'Breslau',
   'Brigantia',
   'Donar III',
   'Evita VI',
   'May-Be 1960',
   '"R.-V. Germania; Leitmeritz"',
   'Surprise']),
 ('MEX',
  ['A North American Team',
   'Acipactli',
   'Chamukina',
   'Mexico',
   'Mexico-1',
   'Mexico-2',
   'Nausikaa 4',
   'Tlaloc',
   'Xolotl']),
 ('ARG',
  ['Acturus',
   'Antares',
   'Arcturus',
   'Ardilla',
   'Argentina',
   'Argentina-1',
   'Argentina-2',
   'Blue Red',
   'Covunco III',
   'Cupidon III',
   'Djinn',
   'Gullvinge',
   'Matrero II',
   'Mizar',
   'Pampero',
   'Rampage',
   'Tango',
   'Wiking']),
 ('AFG', ['Afghanistan'])]

In [14]:
equiposArgentinos= equiposOlimpicosRDD.filter(lambda l: "ARG" in l)
equiposArgentinos.collect()

[['4', 'Acturus', 'ARG'],
 ['37', 'Antares', 'ARG'],
 ['42', 'Arcturus', 'ARG'],
 ['43', 'Ardilla', 'ARG'],
 ['45', 'Argentina', 'ARG'],
 ['46', 'Argentina-1', 'ARG'],
 ['47', 'Argentina-2', 'ARG'],
 ['119', 'Blue Red', 'ARG'],
 ['238', 'Covunco III', 'ARG'],
 ['252', 'Cupidon III', 'ARG'],
 ['288', 'Djinn', 'ARG'],
 ['436', 'Gullvinge', 'ARG'],
 ['644', 'Matrero II', 'ARG'],
 ['672', 'Mizar', 'ARG'],
 ['774', 'Pampero', 'ARG'],
 ['843', 'Rampage', 'ARG'],
 ['1031', 'Tango', 'ARG'],
 ['1162', 'Wiking', 'ARG']]

In [15]:
equiposOlimpicosRDD.countApprox(20)

1185

In [16]:
deportistaOlimpicosRDD = sc.textFile(path+"deportista.csv") \
    .map(lambda l : l.split(","))
deportistaOlimpicosRDD2 = sc.textFile(path+"deportista2.csv") \
    .map(lambda l : l.split(","))

In [17]:
deportistaOlimpicosRDD = deportistaOlimpicosRDD \
    .union(deportistaOlimpicosRDD2)

In [18]:
deportistaOlimpicosRDD.count()

135572

In [19]:
deportistaOlimpicosRDD.top(2)

[['deportista_id', 'nombre', 'genero', 'edad', 'altura', 'peso', 'equipo_id'],
 ['99999', 'Alexander Grant Alick Rennie', '1', '32', '182', '71', '967']]

In [20]:
deportistaOlimpicosRDD.map(lambda l: [l[6], l[:6]]) \
.join(equiposOlimpicosRDD.map(lambda x: [x[0], x[2]])) \
.takeSample(False, 5, 25)

[('970', (['68062', 'Lee MinHui', '2', '28', '174', '65'], 'KOR')),
 ('154', (['39161', 'Angel Merdzhanov Gavrilov', '1', '24', '0', '0'], 'BUL')),
 ('1084',
  (['62843', 'Olha Vasylivna Korobka', '2', '18', '181', '167'], 'UKR')),
 ('678', (['97550', 'Puntsagiin Skhbat', '1', '24', '174', '82'], 'MGL')),
 ('1096', (['106789', 'Hugo Scherzer', '1', '43', '0', '0'], 'USA'))]

In [21]:
resultado = sc.textFile(path+"resultados.csv") \
.map(lambda l: l.split(","))

In [22]:
resultadoGanador = resultado.filter(lambda l : "NA" not in l[1])

In [23]:
resultadoGanador.take(2)

[['resultado_id', 'medalla', 'deportista_id', 'juego_id', 'evento_id'],
 ['4', 'Gold', '4', '2', '4']]

In [24]:
deportistaPais = deportistaOlimpicosRDD \
    .map(lambda l: (l[-1], l[:-1])) \
    .join(
        equiposOlimpicosRDD.map(lambda x: (x[0], x[2]))
    )


In [25]:
deportistaPais.join(resultadoGanador).take(5)

[('74',
  ((['65', 'Patimat Abakarova', '2', '21', '165', '49'], 'AZE'), 'Gold')),
 ('74', ((['129', 'Ruslan Abbasov', '1', '22', '181', '74'], 'AZE'), 'Gold')),
 ('74', ((['130', 'Tural Abbasov', '1', '18', '182', '76'], 'AZE'), 'Gold')),
 ('74', ((['131', 'Tran Abbasova', '2', '33', '159', '53'], 'AZE'), 'Gold')),
 ('74',
  ((['335', 'Abdulqdir Abdullayev', '1', '28', '188', '91'], 'AZE'), 'Gold'))]

In [31]:
valoresMedallas = {'Gold': 7,'Silver' : 5,'Bronze':4}

In [30]:
paisesMedallas = deportistaPais.join(resultadoGanador)

In [29]:
paisesMedallas.map(lambda x:x[1][0][-1])

PythonRDD[66] at RDD at PythonRDD.scala:56

In [33]:
rdd = sc.textFile(path + "deportista.csv")
rdd.first()

'deportista_id,nombre,genero,edad,altura,peso,equipo_id'

In [34]:


rdd2 = sc.textFile(path + "paises.csv")
rdd2.first()



'id,equipo,sigla'

In [35]:


rdd3 = sc.textFile(path + "resultados.csv")
rdd3.first()


'resultado_id,medalla,deportista_id,juego_id,evento_id'

NameError: name 'spark' is not defined